<a href="https://colab.research.google.com/github/ifymifan/IOT-AI/blob/main/Lab5_1_2_Machine_learning_Mllib_Pipline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!pip install pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Titanic data")\
        .getOrCreate()
#Check Spark Session Information
spark # this line was incorrectly indented

In [32]:
from pyspark.sql import SparkSession
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Create a SparkSession
spark = SparkSession.builder.appName("Titanic data").getOrCreate()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
# Read the CSV file
df = (spark.read
     .format("csv")
     .option("header", "true")
     .load("/content/drive/MyDrive/Titanic/train.csv")) # Corrected the path to 'Titanic' with uppercase

In [34]:
df.show(3)

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
only showing top 3 rows



In [35]:
# Read the CSV file
df = (spark.read
     .format("csv")
     .option("header", "true")
     .load("/content/drive/MyDrive/Titanic/train.csv")) # Changed 'titanic' to 'Titanic' to match the actual folder name in Google Drive

In [36]:
(train_df,test_df) = df.randomSplit([0.8,0.2],11)
print("Number of train samples:" + str(train_df.count()))
print("Number of test samples:" + str(test_df.count()))

Number of train samples:703
Number of test samples:188


In [56]:

Sex_indexer = StringIndexer(inputCol="Sex", outputCol="Gender")
Embarked_indexer = StringIndexer(inputCol="Embarked", outputCol="Embarked") # Removed duplicate outputCol

input_cols = ["Pclass", "Gender", "Age", "Fare", "Embarked"] #Removed "Geender" which seems to be a typo
output_col = "features"
assembler = VectorAssembler(inputCols=input_cols, outputCol="features")

# Now RandomForestClassifier should be recognized
dt_model = RandomForestClassifier(labelCol="Survived", featuresCol="features")
output_col = "features"
assembler = VectorAssembler(inputCols=input_cols, outputCol="features")

dt_model = RandomForestClassifier(labelCol="Survived", featuresCol="features")

In [77]:
Sex_indexer = StringIndexer(inputCol="Sex", outputCol="Gender")
Embarked_indexer = StringIndexer(inputCol="Embarked", outputCol="Embarked_index") # Changed outputCol to 'Embarked_index'

input_cols = ["Pclass", "Gender", "Age", "Fare", "Embarked_index"] # Updated input_cols to use 'Embarked_index'
output_col = "features"
assembler = VectorAssembler(inputCols=input_cols, outputCol="features")

# Now RandomForestClassifier should be recognized
dt_model = RandomForestClassifier(labelCol="Survived", featuresCol="features")
output_col = "features"
assembler = VectorAssembler(inputCols=input_cols, outputCol="features")

dt_model = RandomForestClassifier(labelCol="Survived", featuresCol="features")

In [83]:
from pyspark.sql.functions import col, mean
from pyspark.ml import Pipeline
from pyspark.ml.feature import Imputer, StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml.classification import RandomForestClassifier
# Use 'assembler' instead of 'Vector_assembler'
pipeline=Pipeline(stages=[Sex_indexer,Embarked_indexer,assembler,dt_model])
print(train_df)

# Calculate the mean of 'Age'
mean_age = train_df.select(mean(col('Age'))).collect()[0][0]

# Use Imputer to fill missing values in 'Age'
age_imputer = Imputer(inputCol="Age", outputCol="Age").setStrategy("mean").setMissingValue(0)

# Convert 'Pclass' to numeric using OneHotEncoder
pclass_indexer = StringIndexer(inputCol="Pclass", outputCol="PclassIndex")
pclass_encoder = OneHotEncoder(inputCol="PclassIndex", outputCol="PclassEncoded")

# Update the pipeline to include the Imputer and OneHotEncoder for 'Pclass'
pipeline = Pipeline(stages=[Sex_indexer, Embarked_indexer, age_imputer, pclass_indexer, pclass_encoder, assembler, dt_model])

# Update input_cols to include the encoded 'Pclass' column
input_cols = ["PclassEncoded", "Gender", "Age", "Fare", "Embarked_index"]  # Updated input_cols
assembler = VectorAssembler(inputCols=input_cols, outputCol="features") # Redefine assembler with updated input_cols


# Fit and transform the data
train_df = pipeline.fit(train_df).transform(train_df)

# If you still need to fill missing values in 'Fare' and 'Embarked', you can do it after the pipeline:
train_df = train_df.fillna({"Fare": 0, "Embarked": "S"})

DataFrame[PassengerId: string, Survived: string, Pclass: string, Name: string, Sex: string, Age: double, SibSp: string, Parch: string, Ticket: string, Fare: double, Cabin: string, Embarked: string]


IllegalArgumentException: requirement failed: Column Survived must be of type numeric but was actually of type string.

In [82]:
from pyspark.sql.functions import col, mean
from pyspark.ml import Pipeline
from pyspark.ml.feature import Imputer, StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml.classification import RandomForestClassifier

# ... (your existing code) ...

# Convert 'Pclass' to numeric using StringIndexer before OneHotEncoder
pclass_indexer = StringIndexer(inputCol="Pclass", outputCol="PclassIndex")

# Use OneHotEncoder on the indexed 'Pclass' column
pclass_encoder = OneHotEncoder(inputCol="PclassIndex", outputCol="PclassEncoded")

# ... (rest of your existing code) ...

# Update the pipeline to include the StringIndexer and OneHotEncoder for 'Pclass'
pipeline = Pipeline(stages=[Sex_indexer, Embarked_indexer, age_imputer, pclass_indexer, pclass_encoder, assembler, dt_model])

# Update input_cols to include the encoded 'Pclass' column
input_cols = ["PclassEncoded", "Gender", "Age", "Fare", "Embarked_index"]  # Updated input_cols
assembler = VectorAssembler(inputCols=input_cols, outputCol="features") # Redefine assembler with updated input_cols

# ... (rest of your existing code) ...